In [1]:
# this cell enables relative path imports
import os
from dotenv import load_dotenv
load_dotenv()
_PROJECT_PATH: str = os.environ["_project_path"]
_PICKLED_DATA_FILENAME: str = os.environ["_pickled_data_filename"]

import sys
from pathlib import Path
project_path = Path(_PROJECT_PATH)
sys.path.append(str(project_path))

In [2]:
# import all your modules here
import json
import pandas as pd

import config_v2 as cfg
from library_ubidots_v2 import Ubidots as ubi

In [3]:
# set your constants here
baseline=cfg.BASELINE
study=cfg.STUDY

In [4]:
print(baseline)
print(study)

['2023-03-01', '2023-05-31']
['2023-06-01', '2023-07-01']


In [5]:
df_devices = ubi.get_available_devices_v2(label='bancolombia', level='group', page_size=1000)

In [6]:
df_vars = ubi.get_available_variables(list(df_devices['device_id']))

In [7]:
df_vars = df_vars[df_vars['variable_label'].isin(cfg.WHITELISTED_VAR_LABELS)]
VAR_IDS_TO_REQUEST = list(df_vars['variable_id'])
VAR_ID_TO_LABEL = dict(zip(df_vars['variable_id'], df_vars['variable_label']))

In [8]:
CHUNK_SIZE = 10
DATE_INTERVAL_REQUEST = {'start': baseline[0], 'end': study[1]}

df = None
lst_responses = []
n_vars = len(VAR_IDS_TO_REQUEST)
print(f"Making request for the following interval: Baseline:{baseline}, Study:{study}")
for idx in range(0, ubi.ceildiv(len(VAR_IDS_TO_REQUEST), CHUNK_SIZE)):
    idx_start = idx * CHUNK_SIZE
    idx_end = (idx + 1) * CHUNK_SIZE
    chunk = VAR_IDS_TO_REQUEST[idx_start:idx_end]

    

    response = ubi.make_request(
        chunk, 
        DATE_INTERVAL_REQUEST, 
    )

    current_idx = idx_end+1
    
    if (current_idx > n_vars):
        current_idx = n_vars

    print(f"Progress: {100*(current_idx)/n_vars:0.1f}%")

    if response.status_code != 204:

        lst_responses.append(response)
    else: 
        print(f"Empty response for chunk {idx}")

df = ubi.parse_response(lst_responses, VAR_ID_TO_LABEL)


Making request for the following interval: Baseline:['2023-03-01', '2023-05-31'], Study:['2023-06-01', '2023-07-01']
Progress: 3.2%
Progress: 6.0%
Progress: 8.9%
Progress: 11.7%
Progress: 14.6%
Progress: 17.5%
Progress: 20.3%
Progress: 23.2%
Progress: 26.1%
Progress: 28.9%
Progress: 31.8%
Progress: 34.7%
Progress: 37.5%
Progress: 40.4%
Progress: 43.3%
Progress: 46.1%
Progress: 49.0%
Progress: 51.9%
Progress: 54.7%
Progress: 57.6%
Progress: 60.5%
Progress: 63.3%
Progress: 66.2%
Progress: 69.1%
Progress: 71.9%
Progress: 74.8%
Progress: 77.7%
Progress: 80.5%
Progress: 83.4%
Progress: 86.2%
Progress: 89.1%
Progress: 92.0%
Progress: 94.8%
Progress: 97.7%
Progress: 100.0%


In [9]:
pd.to_pickle(df, project_path / 'data'/ _PICKLED_DATA_FILENAME)